<a href="https://colab.research.google.com/github/shoji41/kennkyuusituhaizoku/blob/main/%E7%9C%BC%E7%9E%BC%E7%97%99%E6%94%A3%E8%A9%95%E4%BE%A1%E3%83%A2%E3%83%87%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#step1
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#step2
import os
import json
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import matplotlib.pyplot as plt

In [ ]:
#step3
class FaceLandmarkDataset(Dataset):
    def __init__(self, json_file, img_dir, transform=None):
        with open(json_file, 'r') as f:
            self.data = json.load(f)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img_name = os.path.join(self.img_dir, item['imageName'])

        # --- 追加：画像ファイルが存在するかチェック ---
        if not os.path.exists(img_name):
            # 画像がない場合は None を返す
            return None

        image = cv2.imread(img_name)
        if image is None:
            return None

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 座標データの抽出
        eye = item['leftEye']
        landmarks = np.array([
            eye['pupilVerticalX'],
            eye['upperBrowY'],
            eye['lowerBrowY'],
            eye['upperLidY'],
            eye['lowerLidY']
        ], dtype='float32')

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(landmarks)

# --- 追加：None（画像なしデータ）を無視するための関数 ---
def collate_fn(batch):
    # batchの中から None 以外を抽出する
    batch = list(filter(lambda x: x is not None, batch))
    if len(batch) == 0:
        return torch.Tensor(), torch.Tensor()
    return torch.utils.data.dataloader.default_collate(batch)

In [ ]:
#step4

# 1. 学習済みのResNet18モデルを読み込む
# weights='IMAGENET1K_V1' を使うことで、画像認識に強い初期状態からスタートできます
model = models.resnet18(weights='IMAGENET1K_V1')

# 2. 最終層（全結合層）の書き換え
# ResNet18の元の出力は1000種類（犬、猫など）の分類用ですが、
# 今回は「5つの座標数値（回帰）」を出力するように変更します。
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)

# 3. 計算デバイスの設定（GPUが使えるならGPUを使う）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 4. 損失関数と最適化手法の設定
# 座標のズレを計算するため、平均二乗誤差（MSE）を使用します
criterion = nn.MSELoss()

# Adamという、学習がスムーズに進みやすい最適化アルゴリズムを使います
optimizer = optim.Adam(model.parameters(), lr=0.0001) # 学習率は少し小さめに設定

print(f"使用デバイス: {device}")
print("モデルの準備が完了しました。最終出力層のサイズは 5 です。")

使用デバイス: cuda
モデルの準備が完了しました。最終出力層のサイズは 5 です。


In [ ]:
from tqdm.notebook import tqdm  # 進捗バー用のライブラリをインポート

# --- パス設定 ---
data_configs = [
    {'json': '/content/drive/MyDrive/AI_laboratory_course/眼瞼アプリ/Annotty_export_sample1/blink_annotations.json', 'img': '/content/drive/MyDrive/AI_laboratory_course/眼瞼アプリ/Annotty_export_sample1/images'},
    {'json': '/content/drive/MyDrive/AI_laboratory_course/眼瞼アプリ/Annotty_export_sample2/blink_annotations.json', 'img': '/content/drive/MyDrive/AI_laboratory_course/眼瞼アプリ/Annotty_export_sample2/images'},
    {'json': '/content/drive/MyDrive/AI_laboratory_course/眼瞼アプリ/Annotty_export_sample3/blink_annotations.json', 'img': '/content/drive/MyDrive/AI_laboratory_course/眼瞼アプリ/Annotty_export_sample3/images'}
]

# データセットのリスト作成
datasets = [FaceLandmarkDataset(c['json'], c['img'], transform=transform) for c in data_configs]
combined_dataset = ConcatDataset(datasets)

# DataLoaderの設定（Google Drive読み込み対策として num_workers=2 を推奨しますが、エラーが出るなら0にしてください）
dataloader = DataLoader(combined_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

print(f"学習開始。合計データ数（欠損含む）: {len(combined_dataset)}")

# 学習ループ
num_epochs = 30
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0

    # --- 進捗バーの構築 ---
    # descで現在のエポック数を表示、leave=Falseでエポック終了時にバーを消してスッキリさせます
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

    for images, landmarks in progress_bar:
        if images.numel() == 0:
            continue # バッチが空ならスキップ

        images, landmarks = images.to(device), landmarks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, landmarks)
        loss.backward()
        optimizer.step()

        current_loss = loss.item()
        running_loss += current_loss

        # --- 進捗バーの右側に現在のLossを表示 ---
        progress_bar.set_postfix(loss=f"{current_loss:.6f}")

    # 5エポックごとに平均Lossを表示
    if (epoch + 1) % 1 == 0: # 毎エポック表示するように変更しました（確認しやすいため）
        print(f"Epoch [{epoch+1}/{num_epochs}] Average Loss: {running_loss/len(dataloader):.6f}")

print("学習完了")

学習開始。合計データ数（欠損含む）: 478


Epoch 1/30:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch [1/30] Average Loss: 0.098954


Epoch 2/30:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch [2/30] Average Loss: 0.005039


...

Epoch [28/30] Average Loss: 0.000276


Epoch 29/30:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch [29/30] Average Loss: 0.000289


Epoch 30/30:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch [30/30] Average Loss: 0.000148
学習完了


In [ ]:
def predict_and_visualize(image_path, model, device):
    # 1. モデルを評価モードに設定
    model.eval()

    # 2. 画像の読み込み
    orig_img = cv2.imread(image_path)
    if orig_img is None:
        print(f"画像が見つかりません: {image_path}")
        return

    # 表示用にRGBに変換
    img_rgb = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    h, w, _ = img_rgb.shape

    # 3. 前処理（学習時と同じtransformを適用）
    # transformはステップ3で定義したものを使用します
    input_tensor = transform(img_rgb).unsqueeze(0).to(device)

    # 4. 推論（AIが座標を予測）
    with torch.no_grad():
        preds = model(input_tensor).cpu().numpy()[0]

    # 5. 座標の復元（0.0~1.0の予測値をピクセル単位に変換）
    # preds[0]: pupilVerticalX, preds[1]: upperBrowY, preds[2]: lowerBrowY...
    px = preds[0] * w
    py_coords = preds[1:] * h

    # 6. 結果の描画
    plt.figure(figsize=(10, 10))
    plt.imshow(img_rgb)

    labels = ['Upper Brow', 'Lower Brow', 'Upper Lid', 'Lower Lid']
    colors = ['yellow', 'orange', 'cyan', 'lime']

    # 瞳孔の垂直線（X軸）を引く
    plt.axvline(x=px, color='red', linestyle='--', alpha=0.5, label=f'Pupil X: {px:.1f}')

    # 各Y座標をプロット（すべて予測されたX軸上に配置）
    for i, py in enumerate(py_coords):
        plt.scatter(px, py, s=120, c=colors[i], edgecolors='white',
                    label=f'{labels[i]} (Y: {py:.1f})')

    plt.title("Face Landmark Prediction Result")
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
    plt.axis('off')
    plt.show()

    # 数値データとしても出力
    print(f"--- 予測座標値 (pixel) ---")
    print(f"瞳孔垂直X軸: {px:.2f}")
    for i, py in enumerate(py_coords):
        print(f"{labels[i]}: {py:.2f}")

# --- 実行 ---
# テストしたい画像のパスを1つ選んで指定してください
# 例: セット1の1枚目の画像
test_image_path = '/content/drive/MyDrive/AI_laboratory_course/眼瞼アプリ/Annotty_export_sample1/images/IMG_5053_frame00054.png'

predict_and_visualize(test_image_path, model, device)

画像が見つかりません: /content/drive/MyDrive/AI_laboratory_course/眼瞼アプリ/Annotty_export_sample1/images/IMG_5053_frame00054.png
